In [1]:
# Edgar new data Jan 2024

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from models import *
from utils import *

seed = 2468
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3090'

In [3]:
'''
The imaging is still at 30hz.
The ephys is binarized in 1ms bins.
The speed has corresponding triggers in imaging and ephys.
I.e. for every entry in the speed file, there is a corresponding imaging frame or ephys bin at the given index.
There are files for baseline and for stimulation. In each case, the imaging ROIs and the ephys units are matched.
I.e. index 1 in baseline imaging is the same cell as index 1 is stim imaging.

Please see if this format works for you. Let me know if you want me to downsample all of them to the same frame rate (30Hz) instead if that is easier for you. If this runs ok, then I can run another 1-2 mice through which should put us at 3-4 animals. I will work on getting the combinatorial paradigm in place as well, but it is a little bit complicated right now.
'''
area_ID_ephys = np.load('./data/biological/EB095/area_ID_ephys.npy')
dff_run = np.load('./data/biological/EB095/dff_run.npy')
dff_stim = np.load('./data/biological/EB095/dff_stim.npy')
speed_run = np.load('./data/biological/EB095/speed_run.npy')
speed_triggers_ephys_run = np.load('./data/biological/EB095/speed_triggers_ephys_run.npy')
speed_triggers_img_run = np.load('./data/biological/EB095/speed_triggers_img_run.npy')

FileNotFoundError: [Errno 2] No such file or directory: './data/biological/EB095/speed_triggers_sphys_run.npy'